In [1]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd

import torchvision

In [3]:
from sklearn.linear_model import ElasticNet, Lasso, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV

In [4]:
#Validation function
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [5]:
import data_pipeline as dp
from data_pipeline.data_preprocessor import PreProcessor as pp
# load train set, test set


x_train, y_train, X_test, Y_test = pp("data0520").process()
# print(X_train[0], Y_train[0])

# x_train = Variable(torch.tensor(X_train.value))
# y_train = Variable(torch.tensor(Y_train.value))

   b_type_0  b_type_1     in_ch  3x3_Conv_0  5x5_Conv_0  3x3_ConvDW_0  \
0         0         1 -0.105694   -1.268585    0.034280      0.088689   
1         0         1 -0.105694   -1.552363   -0.968592      1.616244   
2         0         1 -0.105694    1.358030    0.343615     -1.265393   

   5x5_ConvDW_0  3x3_dConv_0  5x5_dConv_0  3x3_dConvDW_0  ...  3x3_ConvDW_14  \
0     -1.428291    -0.765825     1.195483      -0.837409  ...      -1.996510   
1     -0.978944     0.113369     0.932577      -1.040075  ...       0.766441   
2     -1.519004    -1.689102    -1.973204      -0.936835  ...       1.650538   

   5x5_ConvDW_14  3x3_dConv_14  5x5_dConv_14  3x3_dConvDW_14  5x5_dConvDW_14  \
0       0.741349     -1.095785      1.695053        0.142634       -0.516759   
1      -0.499441     -1.870190     -0.421694       -0.098675        2.267808   
2      -1.133263     -0.423677      0.412092        1.322388        0.449442   

   3x3_maxpool_14  3x3_avgpool_14  Identity_14   latency  
0     

In [6]:
# -----------------------------------------------------------------

In [7]:
# lasso regression

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=3))

In [8]:
# ElasticNet regression 

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [9]:
# kernel Ridge regression

KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5) #kernel = 'rbf' , 'sigmoid' 

In [10]:
# Gradient Boosting Regression

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [11]:
# LGB Regressor

model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"


In [ ]:
# XGBoost Algorithm (with GridSearch)

# A parameter grid for XGBoost
params = {'min_child_weight':[i/10.0 for i in range(5,18)], 'gamma':[i/100.0 for i in range(3,6)],  
'subsample':[i/10.0 for i in range(4,9)], 'colsample_bytree':[i/10.0 for i in range(4,8)], 'max_depth': [2,3,4]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=2) 

grid = GridSearchCV(xgb, params)
grid.fit(x_train, y_train)

In [ ]:
# SVR with GridSearch

params = {'gamma' :[i/100.0 for i in range(0,11)],  
          'coef0':[0, 0.1, 0.5, 1], 'C' :[0.1, 0.2, 0.5, 1], 'epsilon':[i/10.0 for i in range(0,6)]}


model_svr = SVR()
grid_search = GridSearchCV(model_svr, params, cv=10, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

#grid_search.best_estimator_

In [ ]:
# Random Forest Regressor with GridSearch


param_grid = [
    {'n_estimators': [3, 10, 30, 60, 90], 'max_features': [50,100,150,200,250,300]},
    {'bootstrap': [True], 'n_estimators': [3, 10, 30, 60, 90], 'max_features': [50,100,150,200,250]},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

#grid_search.best_estimator_

In [ ]:
# -----------------------------------------------------

In [ ]:
# lasso score
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
# ElasticNet score
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
# Kernel Rigde score
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
# Gradient Boosting score
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
# Light GBM score
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
# XGBoost score
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
# SVR score 
score = rmsle_cv(model_svr)
print("SVR score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
# Random Forest score
score = rmsle_cv(regr)
print("SVR score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
# ----------------------prediction---------------------------

In [ ]:
#define a rmsle evaluation function

def rmsle(y, y_pred): 
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
# XGBoost

model_xgb.fit(x_train, y_train)
xgb_train_pred = model_xgb.predict(x_train)
xgb_pred = np.expm1(model_xgb.predict(x_test))
print(rmsle(y_train, xgb_train_pred))

In [ ]:
# LightGBM

model_lgb.fit(x_train, y_train)
lgb_train_pred = model_lgb.predict(x_train)
lgb_pred = np.expm1(model_lgb.predict(x_test.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
GBoost.fit(x_train,y_train)
GB_train_pred = GBoost.predict(x_train)
GB_pred = np.expm1(GBoost.predict(x_test.values))
print(rmsle(y_train, GB_train_pred))

In [ ]:
ENet.fit(train,y_train)
ENet_train_pred = ENet.predict(x_train)
ENet_pred = np.expm1(ENet.predict(x_test.values))
print(rmsle(y_train, ENet_train_pred))

In [ ]:
# ensemble

ensemble = xgb_pred*0.25 + lgb_pred*0.25 + GB_pred*0.5